# 100 Days of Coding – Fantasy Football

## Set Up

In [5]:
import pandas as pd
from os import path
import sqlite3
import nfl_data_py as nfl


# Create database connection
conn = sqlite3.connect('../data/db/database.db')

## Day 1
For the QBs, RBs, and WRs, find the top three seasons by average fantasy points scored per player (standard scoring).  
Then find the player with the highest score per position-season combination.

In [6]:
query = """
    WITH position_season_data AS (
        SELECT
            position,
            s.season,
            ROUND(AVG(fantasy_points),2) AS avg_standard,
            ROUND(AVG(fantasy_points_ppr),2) AS avg_ppr
        FROM seasonal s
        INNER JOIN (
            SELECT
                player_id,
                position
            FROM rosters
            WHERE position IN ('QB', 'WR', 'RB')
        ) AS roster_filtered
            ON roster_filtered.player_id = s.player_id
        WHERE season_type = 'REG'
        GROUP BY position, s.season
    ),
    position_season_rankings AS (
        SELECT
            position,
            season,
            avg_standard,
            avg_ppr,
            RANK() OVER(PARTITION BY position ORDER BY avg_standard DESC) AS rank_standard,
            RANK() OVER(PARTITION BY position ORDER BY avg_ppr DESC) AS rank_ppr
        FROM position_season_data
    ),
    player_season_data AS (
        SELECT
            player_name,
            position,
            s.season,
            ROUND(AVG(fantasy_points),2) AS avg_standard,
            ROUND(AVG(fantasy_points_ppr),2) AS avg_ppr
        FROM seasonal s
        INNER JOIN (
            SELECT
                player_id,
                player_name,
                position
            FROM rosters
            WHERE position IN ('QB', 'WR', 'RB')
        ) AS roster_filtered
            ON roster_filtered.player_id = s.player_id
        WHERE season_type = 'REG'
        GROUP BY player_name, s.season
    ),
    player_season_rankings AS (
        SELECT
            player_name,
            position,
            season,
            avg_standard,
            avg_ppr,
            RANK() OVER(PARTITION BY position, season ORDER BY avg_standard DESC) AS rank_standard,
            RANK() OVER(PARTITION BY position, season ORDER BY avg_ppr DESC) AS rank_ppr
        FROM player_season_data
    )
    SELECT
        pos_sr.position,
        pos_sr.season,
        pos_sr.rank_standard AS rank_avg_standard,
        play_sr.player_name AS player_standard,
        pos_sr.rank_ppr AS rank_avg_ppr,
        play_sr_ppr.player_name AS player_ppr
    FROM position_season_rankings AS pos_sr
    LEFT JOIN (
        SELECT player_name, position, season
        FROM player_season_rankings
        WHERE rank_standard = 1) AS play_sr
        ON play_sr.position = pos_sr.position
            AND play_sr.season = pos_sr.season
    LEFT JOIN (
        SELECT player_name, position, season
        FROM player_season_rankings
        WHERE rank_ppr = 1) AS play_sr_ppr
        ON play_sr_ppr.position = pos_sr.position
            AND play_sr_ppr.season = pos_sr.season
    WHERE pos_sr.rank_standard <= 3;
"""

pd.read_sql(query, conn)


,position,season,rank_avg_standard,player_standard,rank_avg_ppr,player_ppr
0,QB,2013,1,Peyton Manning,1,Peyton Manning
1,QB,2015,2,Cam Newton,3,Cam Newton
2,QB,2016,3,Aaron Rodgers,2,Aaron Rodgers
3,RB,2006,1,LaDainian Tomlinson,1,LaDainian Tomlinson
4,RB,2008,2,DeAngelo Williams,2,DeAngelo Williams
5,RB,2009,3,Chris Johnson,3,Chris Johnson
6,WR,2002,1,Marvin Harrison,1,Marvin Harrison
7,WR,2007,2,Randy Moss,2,Randy Moss
8,WR,2013,3,Josh Gordon,3,Demaryius Thomas
